# Cómo (no) predecir acciones en bolsa

Existen numerosas formas de arruinarte. De entre ellas, crear un algoritmo de machine learning que trate de predecir el valor de las acciones en bolsa para luego invertir tu dinero, es una de las más rápidas y efectivas.

A continuación vamos a ver los distintos motivos por los que no suele ser una buena idea.

# 1. Modelos haciendo trampas

Vamos a simular que somos una persona normal y corriente que está aprendiendo machine learning con Python y un buen día piensa "oye, ya que estoy, por qué no intento predecir las acciones de la bolsa y así me saco un dinero extra". Con la mejor de las intenciones, esta persona se pone manos a la obra.

En primer lugar, lo que haríamos sería elegir una compañia que predecir. Una vez elegida, buscaríamos los datos diarios de cotización en un periodo de tiempo cualquiera. Pongamos entre 2015 y 2021.

In [137]:
import pandas as pd
import yfinance as yf
import datetime

In [138]:
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime(2021, 1, 1)

In [139]:
# Cogeremos Google para nuestro caso de estudio, pero te animo a que repliques todo el experimento con otras acciones
simbolo_de_accion = "GOOG"

In [140]:
valor_acciones = yf.download(simbolo_de_accion, start=start, end=end, progress=False)

In [141]:
valor_acciones.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-02,26.378078,26.490770,26.133251,26.168653,26.168653,28951268
2015-01-05,26.091366,26.144720,25.582764,25.623152,25.623152,41196796
2015-01-06,25.679497,25.738087,24.983908,25.029282,25.029282,57998800
2015-01-07,25.280592,25.292759,24.914099,24.986401,24.986401,41301082
2015-01-08,24.831326,25.105074,24.482782,25.065184,25.065184,67071641


Para simplificar la tarea, intentaríamos quedarnos solo con el valor de cierre diario de la acción (Close). Este se convertirá en el valor a predecir, es decir, nuestro target (y).

In [142]:
valor_acciones = valor_acciones[["Close"]].rename(columns={"Close": "target"})
valor_acciones.head()

,target
Date,
2015-01-02,26.168653
2015-01-05,25.623152
2015-01-06,25.029282
2015-01-07,24.986401
2015-01-08,25.065184


Una vez tenemos la información básica a predecir, el siguiente paso lógico sería buscar qué método de machine learning me permite hacer potentes predicciones de series temporales complejas. Por que claro, ya intuimos que los métodos tradicionales no nos vana a valer. Si no, todo el mundo estaría haciendo lo mismo y ganando mucho dinero, ¿verdad?

Una rápida búsqueda en internet te lleva a las redes neuronales recurrentes, las cuáles son capaces de aprender teniendo en cuenta series de datos de forma recurrente (valga la redundancia). Concretamente nos encontramos con las Long Short-Term Memory (LSTM), un tipo de red que mejora las redes recurrentes tradicionales y es capaz de tener en cuenta tanto datos muy lejanos a los últimos registros como los más recientes.

Suena genial, con las LSTM podremos simplemente tratar de predecir el valor de la acción a cierre de un día en base al valor del día anterior (X). Como estas redes tienen memoria a corto y a largo plazo, aprenderán los patrones de subida y bajada de valor.

Vamos a ello.

In [143]:
# El método shift de pandas desplaza todos los valores n veces (a este desplazamiento se le suele llamar lags)
n_lags = 1
valor_acciones["valor_cierre-1"] = valor_acciones["target"].shift(n_lags)
valor_acciones = valor_acciones.dropna()
valor_acciones.head()

,target,valor_cierre-1
Date,,
2015-01-05,25.623152,26.168653
2015-01-06,25.029282,25.623152
2015-01-07,24.986401,25.029282
2015-01-08,25.065184,24.986401
2015-01-09,24.740576,25.065184


In [144]:
from keras import Sequential
from keras.layers import Dense, LSTM

In [145]:
# Definimos el modelo usando Keras sequential
n_features = 1
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_lags, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

/Users/itortouch/miniconda3/envs/garage_data_pills/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [155]:
# Dejamos un año fuera del entrenamiento para poder evaluar (vamos a excluir por ahora el año 2020)
valor_acciones_train = valor_acciones.loc[:datetime.datetime(2019, 1, 1)]
valor_acciones_test = valor_acciones.loc[datetime.datetime(2019, 1, 1):datetime.datetime(2020, 1, 1)]

In [156]:
# El modelo LSTM espera los inputs en forma [samples, timesteps, features]
X_train = valor_acciones_train["valor_cierre-1"].to_numpy()
X_train = X_train.reshape(X_train.shape[0], 1, 1)
X_train.shape

(1005, 1, 1)

In [157]:
y_train = valor_acciones_train["target"].to_numpy()
y_train.shape

(1005,)

In [158]:
# fit model
model.fit(X_train, y_train, epochs=200, verbose=0)

In [159]:
# Generamos test dataset
X_test = valor_acciones_test["valor_cierre-1"].to_numpy()
X_test = X_test.reshape(X_test.shape[0], 1, 1)
y_test = valor_acciones_test["target"].to_numpy()

# Generar predicciones
y_pred_train = model.predict(X_train).flatten()  # Aplanar en caso de que sea necesario
y_pred_test = model.predict(X_test).flatten()    # Aplanar en caso de que sea necesario

# Preparar los datos para Plotly
# Necesitarás las fechas o índices correspondientes a tus datos de entrenamiento y prueba
indices_train = valor_acciones_train.index
indices_test = valor_acciones_test.index

 1/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step


In [160]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# Calculando MAE y MAPE para el conjunto de entrenamiento
mae_train = mean_absolute_error(y_train, y_pred_train)
mape_train = mean_absolute_percentage_error(y_train, y_pred_train)

# Calculando MAE y MAPE para el conjunto de prueba
mae_test = mean_absolute_error(y_test, y_pred_test)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)

# Imprimir los resultados
print(f'Mean Absolute Error (Train): {round(mae_train,3)}')
print(f'Mean Absolute Percentage Error (Train): {round(mape_train * 100, 3)}%')
print(f'Mean Absolute Error (Test): {round(mae_test,3)}')
print(f'Mean Absolute Percentage Error (Test): {round(mape_test * 100, 3)}%')

Mean Absolute Error (Train): 0.436
Mean Absolute Percentage Error (Train): 1.032%
Mean Absolute Error (Test): 0.592
Mean Absolute Percentage Error (Test): 1.009%


Los resultados son impresionantemente buenos. Vamos a graficarlos a ver qué aspecto tienen.

In [42]:
# Crear el gráfico con Plotly
import plotly.graph_objects as go

fig = go.Figure()

# Agregar las líneas de target y predicciones para entrenamiento
fig.add_trace(go.Scatter(x=indices_train, y=y_train, mode='lines', name='Target Train', line=dict(color='darkturquoise')))
fig.add_trace(go.Scatter(x=indices_train, y=y_pred_train, mode='lines', name='Predicción Train', line=dict(color='burlywood', dash='dot')))

# Agregar las líneas de target y predicciones para prueba
fig.add_trace(go.Scatter(x=indices_test, y=y_test, mode='lines', name='Target Test', line=dict(color='red')))
fig.add_trace(go.Scatter(x=indices_test, y=y_pred_test, mode='lines', name='Predicción Test', line=dict(color='orange', dash='dot')))

# Actualizar el layout para tener un título y etiquetas claras
fig.update_layout(title='Comparación de Target vs. Predicciones',
                  xaxis_title='Fecha',
                  yaxis_title='Valor',
                  legend_title='Leyenda')

# Mostrar el gráfico
fig.show()

¿Veis algo raro? Tomaos tiempo para explorar el gráfico haciendo zoom e intentando entender qué pasa.

## El problema

**OVERFITTING**

In [53]:
# Calculando MAE y MAPE para el conjunto de entrenamiento contra el valor del día anterior en lugar del correspondiente
mae_train = mean_absolute_error(X_train.flatten(), y_pred_train)
mape_train = mean_absolute_percentage_error(X_train.flatten(), y_pred_train)

# Calculando MAE y MAPE para el conjunto de prueba contra el valor del día anterior en lugar del correspondiente
mae_test = mean_absolute_error(X_test.flatten(), y_pred_test)
mape_test = mean_absolute_percentage_error(X_test.flatten(), y_pred_test)

# Imprimir los resultados
print(f'Mean Absolute Error (Train -1 day): {round(mae_train,3)}')
print(f'Mean Absolute Percentage Error (Train -1 day): {round(mape_train * 100, 3)}%')
print(f'Mean Absolute Error (Test -1 day): {round(mae_test,3)}')
print(f'Mean Absolute Percentage Error (Test -1 day): {round(mape_test * 100, 3)}%')

Mean Absolute Error (Train -1 day): 0.131
Mean Absolute Percentage Error (Train -1 day): 0.329%
Mean Absolute Error (Test -1 day): 0.07
Mean Absolute Percentage Error (Test -1 day): 0.123%


Los resultados son prácticamente perfectos. Que los resultados de error con las features (X) sean menores que los de el target (y) es señal inequivoca de overfitting.

Al fin y al cabo, predecir el valor de las acciones es muy complejo y cualquier sistema de optimización/aprendizaje tenderá a hacer esta "trampa" salvo que encuentre algún patrón mejor o forcemos que evite el overfitting.

## Posibles soluciones

Existen muchas posibles soluciones a este problema que pueden usarse de forma independiente o combinadas. Aquí algunos ejemplos:
- Transformar el target en una variable binaria (0 o 1) donde 0 significa que el valor es inferior al día anterior y 1 que el valor es mayor o igual al día anterior.
- Añadir más lags (ventana más grande). e.g. La semana aterior, el mes anterior, tres meses atrás...
- Utilizar modelos menos propensos al overfitting como el Random Forest

# 2. El ciclo económico te engaña

De acuerdo, hemos aprendido de los errores. Vamos a intentar mejorar el modelo aplicando todas las posibles soluciones al problema que hemos tenido previamente.

In [161]:
valor_acciones = valor_acciones.drop(columns=["valor_cierre-1"])

In [166]:
valor_acciones.head()

,target,target_binario
Date,,
2015-01-05,25.623152,False
2015-01-06,25.029282,False
2015-01-07,24.986401,False
2015-01-08,25.065184,True
2015-01-09,24.740576,False


In [163]:
valor_acciones["target_binario"] = valor_acciones["target"] > valor_acciones["target"].shift(1)
valor_acciones

,target,target_binario
Date,,
2015-01-05,25.623152,False
2015-01-06,25.029282,False
2015-01-07,24.986401,False
2015-01-08,25.065184,True
2015-01-09,24.740576,False
...,...,...
2020-12-24,86.942497,True
2020-12-28,88.804497,True
2020-12-29,87.935997,False


In [167]:
def generar_lags(df: pd.DataFrame, n_days_lags: list, target_column_name: str) -> pd.DataFrame:
    df = df.copy()
    for n in n_days_lags:
        df[f'{target_column_name}-lag-{n}'] = df[target_column_name].shift(n)
    return df.dropna()

In [168]:
# Vamos a generar las para la anterior semana completa, el mismo día dos semanas, 4 semanas y tres meses atrás.
valor_acciones_lags = generar_lags(valor_acciones, [1, 2, 3, 4, 5, 6, 7, 7*2, 7*4, 7*4*3], "target_binario")
valor_acciones_lags.head()

,target,target_binario,target_binario-lag-1,target_binario-lag-2,target_binario-lag-3,target_binario-lag-4,target_binario-lag-5,target_binario-lag-6,target_binario-lag-7,target_binario-lag-14,target_binario-lag-28,target_binario-lag-84
Date,,,,,,,,,,,,
2015-05-06,26.211000,False,False,True,True,False,False,False,False,True,False,False
2015-05-07,26.535000,True,False,False,True,True,False,False,False,False,False,False
2015-05-08,26.910999,True,True,False,False,True,True,False,False,True,True,False
2015-05-11,26.785000,False,True,True,False,False,True,True,False,False,False,True
2015-05-12,26.452000,False,False,True,True,False,False,True,True,True,False,False


In [169]:
# Dejamos el último año fuera del entrenamiento para poder evaluar (esta vez no excluimos 2020)
valor_acciones_lags_train = valor_acciones_lags.loc[:datetime.datetime(2020, 1, 1)]
valor_acciones_lags_test = valor_acciones_lags.loc[datetime.datetime(2020, 1, 1):]

In [170]:
from sklearn.ensemble import RandomForestClassifier

In [171]:
X_train = valor_acciones_lags_train.loc[:, "target_binario-lag-1":"target_binario-lag-84"].to_numpy()
y_train = valor_acciones_lags_train["target_binario"].to_numpy()

In [172]:
random_forest_model = RandomForestClassifier(random_state=42, n_estimators=100)
random_forest_model = random_forest_model.fit(X_train, y_train)

In [173]:
X_test = valor_acciones_lags_test.loc[:, "target_binario-lag-1":"target_binario-lag-84"].to_numpy()
y_test = valor_acciones_lags_test["target_binario"].to_numpy()

In [174]:
y_pred_train = random_forest_model.predict(X_train)
y_pred_test = random_forest_model.predict(X_test)

In [175]:
from sklearn.metrics import classification_report

# Calculando el classification report para el conjunto de entrenamiento
cr_train = classification_report(y_train, y_pred_train)

# Calculando el classification report para el conjunto de prueba
cr_test = classification_report(y_test, y_pred_test)

In [176]:
print(cr_train)

              precision    recall  f1-score   support

       False       0.83      0.80      0.82       550
        True       0.83      0.86      0.84       623

    accuracy                           0.83      1173
   macro avg       0.83      0.83      0.83      1173
weighted avg       0.83      0.83      0.83      1173



In [177]:
print(cr_test)

              precision    recall  f1-score   support

       False       0.41      0.43      0.42       105
        True       0.58      0.56      0.57       148

    accuracy                           0.51       253
   macro avg       0.49      0.49      0.49       253
weighted avg       0.51      0.51      0.51       253



Por algún motivo, el error en el test es mucho más alto que en el train dataset. Veamos que aspecto tienen ambos periodos.

In [178]:
indices_train = valor_acciones_lags_train.index
indices_test = valor_acciones_lags_test.index

In [182]:
fig = go.Figure()

# Agregar las líneas de target y predicciones para entrenamiento
fig.add_trace(
    go.Scatter(
        x=indices_train,
        y=valor_acciones_lags_train["target"],
        mode="lines",
        name="Target Train",
        line=dict(color="darkturquoise"),
    )
)

# Agregar las líneas de target y predicciones para prueba
fig.add_trace(
    go.Scatter(
        x=indices_test, y=valor_acciones_lags_test["target"], mode="lines", name="Target Test", line=dict(color="red")
    )
)

# Actualizar el layout para tener un título y etiquetas claras
fig.update_layout(
    title="Explorar evolución del precio (target) por periodos",
    xaxis_title="Fecha",
    yaxis_title="Valor",
    legend_title="Leyenda",
)

# Mostrar el gráfico
fig.show()

Parece que en 2020 hubo algunos movimientos extraños. Pero, más allá, de la baja precisión alcanzada en las predicciones, ¿cómo nos habría afectado económicamente estos errores?

## El problema

In [197]:
# Vamos a calcular cual es la variación media del target cuando este varia negativamente (False) en ambos periodos
valor_acciones_lags_train["target-diff-1"] = valor_acciones_lags_train["target"].diff(1)
valor_acciones_lags_test["target-diff-1"] = valor_acciones_lags_test["target"].diff(1)


/var/folders/jz/_k4hhzds5v19mhfk4vfzc37c0000gn/T/ipykernel_22447/210504948.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/jz/_k4hhzds5v19mhfk4vfzc37c0000gn/T/ipykernel_22447/210504948.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [216]:
variacion_media_negativa_train, variacion_media_positiva_train = (
    valor_acciones_lags_train.query("target_binario == False")["target-diff-1"].mean(),
    valor_acciones_lags_train.query("target_binario == True")["target-diff-1"].mean(),
)
print(
    f"La variación media negativa en train dataset es de {round(variacion_media_negativa_train,3)} y la positiva es de {round(variacion_media_positiva_train,3)}"
)

La variación media negativa en train dataset es de -0.472 y la positiva es de 0.481


In [217]:
variacion_media_negativa_test, variacion_media_positiva_test = (
    valor_acciones_lags_test.query("target_binario == False")["target-diff-1"].mean(),
    valor_acciones_lags_test.query("target_binario == True")["target-diff-1"].mean(),
)
print(
    f"La variación media negativa en test dataset es de {round(variacion_media_negativa_test,3)} y la positiva es de {round(variacion_media_positiva_test,3)}"
)

La variación media negativa en test dataset es de -1.322 y la positiva es de 1.075


In [218]:
variacion_media_negativa_train_fallo, variacion_media_positiva_train_fallo = (
    valor_acciones_lags_train[valor_acciones_lags_train["target_binario"] != y_pred_train]
    .query("target_binario == False")["target-diff-1"]
    .mean(),
    valor_acciones_lags_train.query("target_binario == True")["target-diff-1"].mean(),
)
print(
    f"La variación media negativa en train dataset para aquellos casos en los que el modelo falla es de {round(variacion_media_negativa_train_fallo,3)} y la positiva es de {round(variacion_media_positiva_train_fallo,3)}"
)

La variación media negativa en train dataset para aquellos casos en los que el modelo falla es de -0.487 y la positiva es de 0.481


In [220]:
variacion_media_negativa_test_fallo, variacion_media_positiva_test_fallo = (
    valor_acciones_lags_test[valor_acciones_lags_test["target_binario"] != y_pred_test]
    .query("target_binario == False")["target-diff-1"]
    .mean(),
    valor_acciones_lags_test.query("target_binario == True")["target-diff-1"].mean(),
)
print(
    f"La variación media negativa en test dataset para aquellos casos en los que el modelo falla es de {round(variacion_media_negativa_test_fallo,3)} y la positiva es de {round(variacion_media_positiva_test_fallo,3)}"
)

La variación media negativa en test dataset para aquellos casos en los que el modelo falla es de -1.473 y la positiva es de 1.075


Como podemos observar, el problema aquí está en que la tendencia de las acciones habitualmente va acompasado con el ciclo económico. Por su lado, en el ciclo ecónomico inervienen una cantidad enorme de factores y fenomenos que pueden cambiar en periodos de tiempo muy pequeño. Algunos de los fenómenos que han cambiado radicalmente el ciclo económico fueron:
- La crisis inmobiliaria de 2008
- La crisis del COVID en 2020

Para más inri, dependiendo del tipo de compañía analizada, pueden haber fenómenos específicos de su sector como la burbuja de las .com en los 2000.

En otas ocasiones, los cambios de ciclo o tendencia de una acción pueden ser tan aleatorios como los vividos recientemente con el fenomeno WallStreetBets y GameStop.

Aquí entra la famosa frase de consejo financiero:

`rendimientos pasados no garantizan rendimientos futuros`

## Posibles soluciones

Siendo honesto, este tipo de problemas es genuinamente compejo dada la gran cantidad de variables y fenomenos que, juntos, hacen de este problema uno con casi infinitos grados de libertad. Cuesta imaginar como podríamos estructurar la recolección de datos continua y fiable que nos hubiera permitido anticipar las caidas de las bolsas en marzo de 2020. Sin embargo, si que se pueden desarrollar sistemas que mitiguen o minimicen las perdidas y permitan estrategias que en global ganen dinero. Os animo a que invesigues este camino con mucha prudencia y sin prisa, es un trabajo realmente arduo, si es que es posible.

# Bonus. Intentar entrenar un modelo capaz de predecir distintas acciones

En algun momento de este video quizá hayas podido pensar que la solución a algunos problemas sea entrenar el modelo con muchos datos de distintas acciones, distintos periodos temporales y áreas geográficas. Intuitivamente, cuanto mayor y más variada es la muestra, mejor puede llegar a ser el modelo entrenado para generalizar casos. 

Esto tiene sentido. Pero la cantidad de información y distintas features que habría que conseguir para poder construir un modelo que puediera generalizar de esa forma, sería realmente enorme, hablamos de miles de features. 

Aún en el supuesto de que pudieramos conseguir ese titánico esfuerzo, todavía tendríamos que lidiar con un problema si cabe más complejo que es la maldición de la dimensionalidad. Sin querer entrar en detalles, la maldición de la dimensionalidad se refiere a cuando el train dataset contiene tantas features (dimensiones) que la densidad de datos (muestras) en el espacio generado por el modelo estará practicamente vacío y le costará muchísimo realizar predicciones robustas. 

# Conclusion

La predicción de acciones (stocks) es un problema apasionante, que a muchos nos reta y motiva durante nuestros primeros pasos en la ciencia de datos. Mientras esto no nos obsesione ni nos afecte personalmente en lo económico, es un problema perfecto para progresar poco a poco y comprender multitud de conceptos como:
- Análisis de series temporales
- Procesamiento y transformación de datos
- Sistemas/problemas complejos con alta dimensionalidad
- Modelización (deep learning + machine learning)

Dicho esto, espero que aunque sigas sin saber cómo predecir el valor de las acciones, al menos sepas como NO hacerlo.